In [1]:
import numpy as np
from dataclasses import dataclass, field

In [2]:
test_text = "2333133121414131402"


In [3]:
def get_files_and_spacing(text: str):
    arr = np.array([int(x) for x in text])
    return arr[::2], arr[1::2]

In [4]:
with open("input.txt") as f:
    data_text = f.readlines()
data_text = data_text[0].replace("\n", "")

In [5]:
def calc_check_sum(disk_space: list[int]) -> int:
    check_sum = 0
    for i, id in enumerate(disk_space):
        if id == -1:
            continue
        check_sum += i * id
    return check_sum

### Part one

In [ ]:
files, spaces = get_files_and_spacing(data_text)
first_line = files[0]
remaining_files = files[1:]
id = 0
disk_space = [id] * first_line

for file, space in zip(remaining_files, spaces):
    id += 1
    disk_space += [-1] * space
    disk_space += [id] * file
    
back_index = len(disk_space) - 1
front_idx = 0
while front_idx < back_index:
    if disk_space[front_idx] == -1:
        if disk_space[back_index] == -1:
            back_index -= 1
            continue
        disk_space[front_idx], disk_space[back_index] = disk_space[back_index], disk_space[front_idx]
    else:
        front_idx += 1
# print(disk_space)
calc_check_sum(disk_space)

### Part two

In [ ]:
@dataclass
class MemElement:
    size: int
    is_chunk: bool
    values: list[int]
    
    def is_empty(self):
        return len(self.values) == 0
    
    def get_free_space(self):
        return self.size - len(self.values)
    
    def move_element(self, other):
        self.values += other.values
        other.values.clear()
    def __repr__(self):
        value_list = self.values + self.get_free_space() * [0]
        loc_repr = "".join([str(i) for i  in value_list])
        return loc_repr
    
    def get_memory(self):
        return self.values + self.get_free_space() * [-1]

In [ ]:
files, spaces = get_files_and_spacing(data_text)
first_line = files[0]
remaining_files = files[1:]
id = 0
disk_space = []
disk_space.append(MemElement(int(first_line), True, [0] * first_line))
new_disk_space = disk_space.copy()
for file, space in zip(remaining_files, spaces):
    id += 1
    disk_space.append(MemElement(int(space), False, []))
    disk_space.append(MemElement(int(file), True, [id] * file))
    
back_index = len(disk_space) - 1
front_idx = 0
for back_idx in range(back_index, 0, -1):
    if not disk_space[back_idx].is_chunk:
        continue
    for front_idx in range(1, back_idx):
        if disk_space[front_idx].get_free_space() >= disk_space[back_idx].size:
            disk_space[front_idx].move_element(disk_space[back_idx])
memory = []
for chunk in disk_space:
    memory += chunk.get_memory()
calc_check_sum(memory)